In [317]:
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from adjustText import adjust_text
import math
import json

class OperonFileReader:
    def __init__(self, file_path):
        self.file_path = file_path
    
    def read_operons(self):
        operons = []
        with open(self.file_path, 'r') as f:
            lines = f.readlines()
            operon = None
            for line in lines:
                
                fields = line.split('\t')
                if fields[0] == 'Operon':
                    continue  # skip header line
                elif fields[0].strip().isdigit():
                    if operon is not None:
                        operons.append(operon)
                    operon = {'id': int(fields[0].strip()), 'genes': []}
                elif operon is not None:
                    gene_fields = fields
                    if len(gene_fields) == 2:
                        continue
                    
                    gene = {
                        'id': gene_fields[1],
                        'type': gene_fields[2],
                        'cog_gene': gene_fields[3],
                        'pos_left': int(gene_fields[4]) if fields[4].isdigit() else None,
                        'pos_right': int(gene_fields[5]) if fields[5].isdigit() else None,
                        'strand': gene_fields[6],
                        'function': ' '.join(gene_fields[7:]) if len(gene_fields) > 7 else None
                    }
                    operon['genes'].append(gene)
            if operon is not None:
                operons.append(operon)
        return operons
    
    @classmethod
    def find_operons_by_keywords(cls, file_path, keywords):
        reader = cls(file_path)
        operons = reader.read_operons()
        
        result = []
        for operon in operons:
            for gene in operon['genes']:
                if gene['function'] is not None:
                    for keyword in keywords:
                        if keyword in gene['function'].lower():
                            result.append(operon)
                            break
                    else:
                        continue
                    break
        
        return result

chatGPT: i give you a file contains multiple operons with genes relevant to carbon/nitorgen/phosphorus, please tell me what operon has two of them to show co-regulation information:

In [318]:
def adjust_annotations(annotations, xlim, y_range):
    normlization = 10
    # Calculate the length of the x-axis
    x_length = xlim[1] - xlim[0]

    # Initialize a list to store adjusted annotations
    new_annotations = []

    # Loop through each annotation in the input list
    for i, ann in enumerate(annotations):
        if ann['text'] == "NA\n" or ann['text'] == "NA":
            
            new_annotations.append(ann)
            continue
        # Check if this annotation overlaps with any previous annotations
        overlap = False
        for prev_ann in new_annotations:
            if prev_ann['text']=="NA\n" or prev_ann['text']=="NA":
                continue
            if abs(prev_ann['x'] - ann['x']) < x_length / normlization and abs(prev_ann['y'] - ann['y']) < y_range:
                overlap = True
                #print("overlapped",ann['text'],"previous",prev_ann['text'])
                break
        
        # If there's an overlap, adjust the y-coordinate of this annotation
        if overlap:
            ann['x'] += x_length / normlization
            
            # Check if the adjusted annotation overlaps with any previous annotations
            while True:
                overlap = False
                for prev_ann in new_annotations:
                    if prev_ann['text']=="NA\n" or prev_ann['text']=="NA":
                        continue
                    if abs(prev_ann['x'] - ann['x']) < x_length / normlization and abs(prev_ann['y'] - ann['y']) < y_range:
                        overlap = True
                        break
                if overlap:
                    ann['x'] += x_length / normlization
                else:
                    break
        
        # Add this annotation to the new list
        new_annotations.append(ann)
        
    # Return the adjusted list of annotations
    return new_annotations


In [329]:
class OperonVisualizer:
    def __init__(self, operons):
        self.operons = operons
    
    def visualize(self):
        fig = go.Figure()
        colors = ['red', 'green', 'blue', 'purple', 'orange', 'pink', 'gray']
        max_right = 0
        annotations = [] # for text
        annotations2 = [] # for +/-
        i = 0 # y axis count
        
        for operon in self.operons:
            i += 1
            num_genes = len(operon['genes'])
            if num_genes <= 1:
                i -= 1
                continue

            duplicates = [] # remove duplicate functions within each operon
            for j, gene in enumerate(operon['genes']):
                if gene['function'] not in duplicates:
                    duplicates.append(gene['function'])
                else:
                    gene['function'] = '' # hide text if duplicated functions

                if gene['pos_right'] is not None and gene['pos_right'] > max_right:
                    max_right = gene['pos_right']
                if gene['pos_left'] is not None and gene['pos_right'] is not None:
                    fig.add_shape(
                        type='line',
                        x0=gene['pos_left'],
                        y0=i,
                        x1=gene['pos_right'],
                        y1=i,
                        line=dict(color=colors[i%len(colors)], width=5),
                    )
                    
                    if num_genes > 1:
                        xanchor = 'center'
                        if j == 0:
                            xanchor = 'right'
                        elif j == num_genes - 1:
                            xanchor = 'left'
                    else:
                        xanchor = 'center'
                    
                    # Split the function text into multiple lines based on the maximum number of characters per line
                    max_char_per_line = 25
                    function_text = gene['function'] if gene['function'] is not None else ""
                    function_lines = []
                    line = ""
                    for word in function_text.split():
                        if len(line + " " + word) <= max_char_per_line:
                            line += " " + word
                        else:
                            function_lines.append(line.strip())
                            line = word
                    function_lines.append(line.strip())
                    
                    # Calculate the padding based on the number of lines of the text
                    padding = 0.2 + (len(function_lines) - 1) * 0.08
                    midpoint = (gene['pos_left'] + gene['pos_right'])/2

                    if gene['function']=="NA\n":
                        annotations.append(dict(
                            x=midpoint,
                            y=i-padding,
                            text='<br>'.join(function_lines),
                            showarrow=False,
                            font=dict(color='black', size=12),
                            xanchor=xanchor,
                        ))
                    else:
                        annotations.append(dict(
                            x=midpoint,
                            y=i+padding,
                            text='<br>'.join(function_lines),
                            showarrow=False,
                            font=dict(color='black', size=12),
                            xanchor=xanchor,
                        ))
                    

                    # Add +/- symbols to indicate the strand of the DNA
                    
                    if gene['strand'] == '+':
                        annotations2.append(dict(
                            x=midpoint,
                            y=i,
                            text='+',
                            showarrow=False,
                            font=dict(color='black', size=12),
                            xanchor='left',
                        ))
                    elif gene['strand'] == '-':
                        annotations2.append(dict(
                            x=midpoint,
                            y=i,
                            text='-',
                            showarrow=False,
                            font=dict(color='black', size=12),
                            xanchor='left',
                        ))
                   
        h = 500 + 50*len(self.operons)     
        
        fig.update_layout(
            
            xaxis=dict(
                title='Position on Genome',
                range=[0, max_right + 100],
            ),
            yaxis=dict(
                title='Operons',
                tickmode='linear',
                dtick=1,
                range=[0, i+1],
            ),
            height=h,
            width=h*0.85,
        )
        
        # manual fint-tuning
        
        annotations = adjust_annotations(annotations,fig.layout.xaxis.range,0.2)
        annotations += annotations2
        fig.update_layout(annotations=annotations)
        
        fig.show()
        return fig,annotations


In [330]:
def split_operons(operons, max_operons_per_fig):
    num_operons = len(operons)
    num_figs = math.ceil(num_operons / max_operons_per_fig)
    operon_chunks = [operons[i:i+max_operons_per_fig] for i in range(0, num_operons, max_operons_per_fig)]
    return operon_chunks




In [331]:
p18 = "./data/cropps_summer_tax/18/operon/list_of_operons_296772"
p22 = "./data/cropps_summer_tax/22/680027/list_of_operons_680027"

def plot_operon(path,title,kw=['phos', 'carbon', 'nitro']):
    reader = OperonFileReader(path)
    operons = reader.find_operons_by_keywords(path,kw)
    with open(path+".json","w") as f:
        json.dump(operons,f)
    # Define the maximum number of operons per figure
    max_operons_per_fig = 20

    # Split the operons into chunks
    operon_chunks = split_operons(operons, max_operons_per_fig)

    # Iterate over the operon chunks and create a separate figure for each chunk
    for i, operon_chunk in enumerate(operon_chunks):
        fig,annotations = OperonVisualizer(operon_chunk).visualize()
        fig.write_image(f'operon_visualizer_{title}_{i+1}.png')
    return annotations
    

In [332]:
ann = plot_operon(p18,"Maize18")

In [333]:
ann = plot_operon(p22,"Maize22")